In [1]:
import sys
sys.path.append('../')
import pandas as pd
import numpy as np
from roverdata.db import DataWarehouse
from main import main
from taxes_increase import utils

## Get Data

In [2]:
#LTV, retrans = main()

In [3]:
LTV = pd.read_csv('../LTV.csv')

In [4]:
LTV.head()

,requester_id,service,new_repeat,new_account,first_observed_stay_added,dates,stays,nrt,gmv
0,5726476,*,repeat,old,2018-08-25 02:40:16.000000,360,4,294.30,1400.0
1,7312458,dog-walking,new,old,2018-08-25 13:47:58.000000,180,25,355.05,1315.0
2,7192616,drop-in,new,old,2018-08-26 23:24:12.000000,360,21,125.01,463.0
3,1426949,dog-walking,repeat,old,2018-08-25 11:30:23.000000,30,5,34.02,126.0
4,5441756,*,repeat,old,2018-08-27 20:06:46.000000,180,7,37.80,140.0


In [5]:
LTV.shape

(811965, 9)

In [6]:
retrans = pd.read_csv('../retrans.csv')

In [7]:
retrans.head()

,new_repeat,new_account,service,num_needs,num_booked,ntb14,num_owners,reneeded14,rebooked14
0,new,old,dog-walking,4750,1759,0.370316,1307,289,183
1,new,old,drop-in,7155,3246,0.453669,2271,266,165
2,repeat,old,overnight,131589,94585,0.718791,58484,15414,12696
3,repeat,old,*,279234,217808,0.780020,95336,34804,30414
4,repeat,old,dog-walking,59209,50365,0.850631,17613,11249,10365


## Calculate Percent GMV Drop for Breakeven

In [8]:
fee=pd.Series([.035,.07],name="Fee Increase")
tax = pd.Series('No tax/4% tax/8% tax'.split('/'),name='Sales Tax')
#pd.DataFrame(columns=['Relative Boost to Rover take','Breakeven loss to GMV'])
x=pd.DataFrame(fee).assign(tag=1)
x=pd.DataFrame(tax).assign(tag=1).merge(x).drop(columns='tag').assign(tax=[0,0,4,4,8,8])
x['Relative Boost to Rover take']=x['Fee Increase']/(27-x.tax)*100
x['Breakeven loss to GMV']=(1+x['Relative Boost to Rover take'])**-1-1
x['Relative Boost to Rover take']=x['Relative Boost to Rover take'].apply('{:.1%}'.format)
x['Breakeven loss to GMV']=x['Breakeven loss to GMV'].apply('{:.1%}'.format)
x=x.drop(columns='tax').pivot(index='Fee Increase',columns='Sales Tax')
x

Relative Boost to Rover take               Breakeven loss to GMV  \
Sales Tax                          4% tax 8% tax No tax                4% tax   
Fee Increase                                                                    
0.035                               15.2%  18.4%  13.0%                -13.2%   
0.070                               30.4%  36.8%  25.9%                -23.3%   

                              
Sales Tax     8% tax  No tax  
Fee Increase                  
0.035         -15.6%  -11.5%  
0.070         -26.9%  -20.6%

## Mean and Bootstrapped Standard Error

In [9]:
LTV180 = LTV[LTV.dates==180].drop(columns=['dates','requester_id'])
LTV180_new = LTV180[LTV180.new_repeat=='new']
LTV180_new_account = LTV180[LTV180.new_account=='new']
splitter = ['service']

In [10]:
np.random.seed(9)
boots_new = utils.bootstrap(LTV180_new,samplesize=10000,nsamples=1000).groupby('service sample_id'.split()).mean()
means_new = boots_new.groupby(splitter).mean().round(2)
means_new

,stays,nrt,gmv
service,,,
*,2.09,37.26,143.86
dog-walking,4.87,62.07,235.60
drop-in,2.89,43.10,164.43
overnight,1.15,29.36,114.79


In [13]:
np.random.seed(9)
boots_new_account = utils.bootstrap(LTV180_new_account,samplesize=10000,nsamples=1000).groupby('service sample_id'.split()).mean()
means_new_account = boots_new.groupby(splitter).mean().round(2)
means_new_account

,stays,nrt,gmv
service,,,
*,2.09,37.26,143.86
dog-walking,4.87,62.07,235.60
drop-in,2.89,43.10,164.43
overnight,1.15,29.36,114.79


In [15]:
var10k_new = boots_new.groupby(splitter).var()
propSE10k_new = var10k_new**.5/means_new
propSE10k_new

,stays,nrt,gmv
service,,,
*,0.023231,0.021683,0.021945
dog-walking,0.016069,0.018148,0.018227
drop-in,0.019986,0.020391,0.020582
overnight,0.021134,0.020875,0.021464


In [17]:
var10k_new_account = boots_new_account.groupby(splitter).var()
propSE10k_new_account = var10k_new_account**.5/means_new_account
propSE10k_new_account

,stays,nrt,gmv
service,,,
*,0.022855,0.021585,0.021858
dog-walking,0.016430,0.019087,0.019190
drop-in,0.020095,0.020554,0.020695
overnight,0.020837,0.020849,0.021612


## Get Customer Volume

In [28]:
retrans_new = retrans.drop(columns=['new_account']).groupby(['new_repeat', 'service']).sum()
retrans_new.reset_index(inplace=True)
retrans_new = retrans_new.loc[retrans_new['new_repeat']=="new"]
retrans_new

,new_repeat,service,num_needs,num_booked,ntb14,num_owners,reneeded14,rebooked14
0,new,*,110360,49562,0.883670,36150,3255,1572
1,new,dog-walking,12678,5057,0.786310,3766,840,576
2,new,drop-in,21690,10905,0.980604,6403,743,485
3,new,overnight,65792,29430,0.884265,24635,2107,1111


### New Customers GMV

In [29]:
utils.maketable(df=retrans_new, means=means_new, var10k=var10k_new, splitter=splitter, thing='gmv')

,30 Day Owner Volume,average gmv,sample_size05,sample_size02,30 day detectable
service,,,,,
*,36150,143.86,7400,46250,8.1%
dog-walking,3766,235.60,5105,31907,20.9%
drop-in,6403,164.43,6509,40683,18.1%
overnight,24635,114.79,7079,44246,9.6%


### New Customers Stays

In [34]:
utils.maketable(df=retrans_new, means=means_new, var10k=var10k_new, splitter=splitter, thing='stays')

,30 Day Owner Volume,average stays,sample_size05,sample_size02,30 day detectable
service,,,,,
*,36150,2.09,8292,51828,8.6%
dog-walking,3766,4.87,3967,24799,18.4%
drop-in,6403,2.89,6137,38362,17.6%
overnight,24635,1.15,6863,42895,9.5%


In [31]:
retrans_new_account = retrans.groupby(['new_repeat', 'new_account', 'service']).sum()
retrans_new_account.reset_index(inplace=True)
retrans_new_account = retrans_new_account.loc[retrans_new_account['new_account']=="new"]
retrans_new_account

,new_repeat,new_account,service,num_needs,num_booked,ntb14,num_owners,reneeded14,rebooked14
0,new,new,*,72403,33674,0.465091,22334,2087,1124
1,new,new,dog-walking,7928,3298,0.415994,2459,551,393
2,new,new,drop-in,14535,7659,0.526935,4132,477,320
3,new,new,overnight,43613,19977,0.458051,14797,1262,713


### New Accounts GMV

In [33]:
utils.maketable(df=retrans_new_account, means=means_new_account, var10k=var10k_new_account, splitter=splitter, thing='gmv')

,30 Day Owner Volume,average gmv,sample_size05,sample_size02,30 day detectable
service,,,,,
*,22334,143.86,7341,45885,10.3%
dog-walking,2459,235.60,5658,35368,27.3%
drop-in,4132,164.43,6581,41132,22.7%
overnight,14797,114.79,7177,44858,12.5%


### New Accounts Stays

In [35]:
utils.maketable(df=retrans_new_account, means=means_new_account, var10k=var10k_new_account, splitter=splitter, thing='stays')

,30 Day Owner Volume,average stays,sample_size05,sample_size02,30 day detectable
service,,,,,
*,22334,2.09,8026,50165,10.8%
dog-walking,2459,4.87,4148,25925,23.3%
drop-in,4132,2.89,6204,38780,22.0%
overnight,14797,1.15,6671,41697,12.1%
